In [1]:
from nn_ansatz import setup
from nn_ansatz import run_vmc

ModuleNotFoundError: No module named 'nn_ansatz'

In [ ]:
# using routines

lr, damping, nc = 1e-4, 1e-4, 1e-4
n_pre_it = 500
n_walkers = 512
n_layers = 2
n_sh = 64
n_ph = 16
n_det = 8
n_it = 1000
seed = 1


config = setup(system='Be',
               n_pre_it=500,
               n_walkers=512,
               n_layers=2,
               n_sh=64,
               n_ph=16,
               opt='kfac',
               n_det=8,
               print_every=1,
               save_every=5000,
               lr=lr,
               n_it=1000,
               norm_constraint=nc,
               damping=damping)



In [ ]:
run_vmc(**config)

In [ ]:
# using core functions 

key = rnd.PRNGKey(seed)

mol = SystemAnsatz(r_atoms,
                   z_atoms,
                   n_el,
                   n_el_atoms=n_el_atoms,
                   n_layers=n_layers,
                   n_sh=n_sh,
                   n_ph=n_ph,
                   n_det=n_det,
                   step_size=step_size)

wf, kfac_wf, wf_orbitals = create_wf(mol)
params = initialise_params(key, mol)
d0s = expand_d0s(initialise_d0s(mol), n_walkers)

sampler, equilibrate = create_sampler(wf, mol, correlation_length=10)

if load_pretrain:
    params, walkers = load_pk(pre_path)
    walkers = mol.initialise_walkers(walkers=walkers,
                                     n_walkers=n_walkers,
                                     equilibrate=equilibrate,
                                     params=params,
                                     d0s=d0s)
else:
    walkers = mol.initialise_walkers(n_walkers=n_walkers)
    params, walkers = pretrain_wf(params,
                                  wf,
                                  wf_orbitals,
                                  mol,
                                  walkers,
                                  n_it=n_pre_it,
                                  lr=pre_lr,
                                  n_eq_it=n_pre_it)

grad_fn = create_grad_function(wf, mol)

if opt == 'kfac':
    update, get_params, kfac_update, state = kfac(kfac_wf, wf, mol, params, walkers, d0s,
                                                  lr=lr,
                                                  damping=damping,
                                                  norm_constraint=norm_constraint)
else:
    init, update, get_params = adam(lr)
    state = init(params)

steps = trange(0, n_it, initial=0, total=n_it, desc='training', disable=None)
for step in steps:
    key, subkey = rnd.split(key)

    walkers, acceptance, step_size = sampler(params, walkers, d0s, subkey, step_size)

    grads, e_locs = grad_fn(params, walkers, d0s)

    if opt == 'kfac':
        grads, state = kfac_update(step, grads, state, walkers, d0s)

    state = update(step, grads, state)
    params = get_params(state)

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')